### Q1. Running Elastic

Run Elastic Search 8.17.6, and get the cluster information. If you run it on localhost, this is how you do it:

Get the docker image
```
    docker run -it \
    --rm \
    --name elasticsearch \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    elasticsearch:8.17.6
```

In [1]:
!curl localhost:9200

{
  "name" : "1ab3d33fd54e",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "kXajB-LsTlur1Jg1La1uRQ",
  "version" : {
    "number" : "8.17.6",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "dbcbbbd0bc4924cfeb28929dc05d82d662c527b7",
    "build_date" : "2025-04-30T14:07:12.231372970Z",
    "build_snapshot" : false,
    "lucene_version" : "9.12.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   541  100   541    0     0  21614      0 --:--:-- --:--:-- --:--:-- 23521


In [2]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200')

version.build_hash = dbcbbbd0bc4924cfeb28929dc05d82d662c527b7

In [3]:
es_client.info()['version']['build_hash']

'dbcbbbd0bc4924cfeb28929dc05d82d662c527b7'

In [4]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

## Q2. Indexing the data

In [5]:
# pip install elasticsearch==8

In [6]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch("http://localhost:9200")

In [7]:
index_setting = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_setting)

C:\Users\crab\AppData\Local\Temp\ipykernel_33512\311084524.py:18: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es_client.indices.create(index=index_name, body=index_setting)


BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [course-questions/0hakLEihSrGW5G34PNuXDw] already exists')

In [8]:
from tqdm.auto import tqdm

In [9]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document = doc)

  0%|          | 0/948 [00:00<?, ?it/s]

Which function do you use for adding your data to elastic?

- index

## Q3. Searching


In [10]:
query = "How do execute a command on a Kubernetes pod?"

In [11]:
def elastic_search(query):

    search_query ={
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    } 
                }
            }
        }
    }
    
    response = es_client.search(index=index_name, body=search_query)
    
    #result_docs = []
    
    #for hit in response['hits']['hits']:
    #    result_docs.append(hit['_source'])
    
    return response

In [12]:
response = elastic_search(query)

C:\Users\crab\AppData\Local\Temp\ipykernel_33512\484972523.py:18: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es_client.search(index=index_name, body=search_query)


In [13]:
response['hits']['hits'][0]['_score']

45.04243

What's the score for the top ranking result?

- `44.5`

## Q4. Filtering


In [14]:
query = "How do copy a file to a Docker container?"

In [29]:
def elastic_search(query):

    search_query ={
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text"],
                        "type": "best_fields"
                    } 
                },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
            }
        }
    }
    
    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
         result_docs.append(hit['_source'])
    
    return result_docs

In [34]:
result_docs = elastic_search(query)

result_docs[2]['question']

C:\Users\crab\AppData\Local\Temp\ipykernel_33512\4189531011.py:23: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es_client.search(index=index_name, body=search_query)


'How do I debug a docker container?'

Return 3 results. What's the 3rd question returned by the search engine?

- `How do I debug a docker container?`

## Q5. Building a prompt

In [17]:
context_template = """
Q: {question}
A: {text}
""".strip()

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [18]:
 context_pieces = []

for hit in response['hits']['hits']:
    doc = hit['_source']
    context_piece = context_template.format(**doc)
    context_pieces.append(context_piece)

context = '\n\n'.join(context_pieces)

In [19]:
prompt = prompt_template.format(question=query, context=context)

In [20]:
len(prompt)

1458

What's the length of the resulting prompt? (use the len function)

- `1446`

## Q6. Tokens

In [ ]:
# pip install tiktoken

In [23]:
import tiktoken

In [24]:
print(prompt[:100])

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.



In [25]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [26]:
len(encoding.encode(prompt))

327

Use the encode function. How many tokens does our prompt have?
 
- `320`
 

In [27]:
encoding.decode_single_token_bytes(63842)


b"You're"

## Bonus: generating the answer (ungraded)


In [37]:
from openai import OpenAI

In [39]:
import os
from dotenv import load_dotenv

load_dotenv ()

api_key = os.getenv("OPENAI_API_KEY")
print(api_key[:4])

sk-p


In [40]:
def llm(prompt):
    responses = client.chat.completions.create(
    model = 'gpt-4.1-nano',
    messages = [{"role": "user", "content" :prompt}]
    )
    
    return responses.choices[0].message.content

In [42]:
client = OpenAI(api_key = api_key)

In [43]:
result = llm(prompt)

In [44]:
result

'The provided FAQ does not contain information on how to copy a file to a Docker container.'

## Bonus: calculating the costs (ungraded)

## 💰 GPT-4o Cost Estimate for 1000 Requests

### 📋 Assumptions:
- **Input tokens per request**: 150  
- **Output tokens per request**: 250  
- **Number of requests**: 1000  
- **GPT-4o Pricing (as of June 17, 2025):**
  - **Input**: $0.005 per 1,000 tokens \n
  - **Output**: $0.015 per 1,000 tokens

---

### 🧮 Calculations:

#### 1. Total Input Tokens:
`150 tokens/request × 1000 requests = 150,000 tokens`

#### 2. Total Output Tokens:
`250 tokens/request × 1000 requests = 250,000 tokens`

#### 3. Input Cost:
`150,000 ÷ 1,000 × $0.005 = $0.75`

#### 4. Output Cost:
`250,000 ÷ 1,000 × $0.015 = $3.75`

### ✅ Total Estimated Cost::
`$0.75 (input) + $3.75 (output) = $4.50`